In [3]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"  # prevents CPU spikes

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
data_dir = "dataset"
img_size = 300
batch_size = 4


In [5]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7,1.3]
)

train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 4418 images belonging to 5 classes.
Found 1103 images belonging to 5 classes.


In [7]:
base_model = EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(img_size, img_size, 3)
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_data.num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [6]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

callbacks = [checkpoint]


Phase 1 Training (15 epochs)

In [7]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15


1105/1105 [==============================] - ETA: 0s - loss: 0.9640 - accuracy: 0.6722
Epoch 1: val_accuracy improved from -inf to 0.62829, saving model to best_model.h5


c:\Users\tsidh\anaconda3\envs\wheat\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1105/1105 [==============================] - 389s 347ms/step - loss: 0.9640 - accuracy: 0.6722 - val_loss: 2.0077 - val_accuracy: 0.6283
Epoch 2/15
1105/1105 [==============================] - ETA: 0s - loss: 0.6038 - accuracy: 0.7875
Epoch 2: val_accuracy improved from 0.62829 to 0.63373, saving model to best_model.h5
1105/1105 [==============================] - 514s 465ms/step - loss: 0.6038 - accuracy: 0.7875 - val_loss: 2.1378 - val_accuracy: 0.6337
Epoch 3/15
1105/1105 [==============================] - ETA: 0s - loss: 0.4931 - accuracy: 0.8280
Epoch 3: val_accuracy improved from 0.63373 to 0.64098, saving model to best_model.h5
1105/1105 [==============================] - 374s 338ms/step - loss: 0.4931 - accuracy: 0.8280 - val_loss: 2.2081 - val_accuracy: 0.6410
Epoch 4/15
1105/1105 [==============================] - ETA: 0s - loss: 0.4528 - accuracy: 0.8486
Epoch 4: val_accuracy improved from 0.64098 to 0.64370, saving model to best_model.h5
1105/1105 [==========================

Phase 2 (fine-tuning)

In [8]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15
1105/1105 [==============================] - ETA: 0s - loss: 0.7314 - accuracy: 0.7549
Epoch 1: val_accuracy did not improve from 0.65639
1105/1105 [==============================] - 1362s 1s/step - loss: 0.7314 - accuracy: 0.7549 - val_loss: 2.1443 - val_accuracy: 0.6491
Epoch 2/15
1105/1105 [==============================] - ETA: 0s - loss: 0.5978 - accuracy: 0.8072
Epoch 2: val_accuracy did not improve from 0.65639
1105/1105 [==============================] - 1328s 1s/step - loss: 0.5978 - accuracy: 0.8072 - val_loss: 2.0402 - val_accuracy: 0.6419
Epoch 3/15
1105/1105 [==============================] - ETA: 0s - loss: 0.4797 - accuracy: 0.8450
Epoch 3: val_accuracy did not improve from 0.65639
1105/1105 [==============================] - 1305s 1s/step - loss: 0.4797 - accuracy: 0.8450 - val_loss: 2.0993 - val_accuracy: 0.6500
Epoch 4/15
1105/1105 [==============================] - ETA: 0s - loss: 0.4287 - accuracy: 0.8615
Epoch 4: val_accuracy did not improve from 0.65639